In [41]:
import pronouncing
import copy
import spacy
from textblob import TextBlob
import gensim
import re
from collections import Counter
import pronouncing
import textstat
import pandas as pd
import string
import re
import pickle
from xgboost import XGBClassifier
fimport numpy as np
import random
nlp = spacy.load('en_core_web_lg')


In [7]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = simple_preprocess(str(sent), deacc=True)
        yield(sent)

In [4]:
def get_topics(num_topics, sentences):
    common_dictionary = Dictionary(common_texts)
    common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
    lda = gensim.models.ldamodel.LdaModel(common_corpus, num_topics=num_topics)
    data_words = list(sent_to_words(sentences))
    curr_corpus = [common_dictionary.doc2bow(text) for text in data_words]
    lda.update(curr_corpus)
    vectors = []
    for doc in other_corpus:
        vector = lda[doc]
        vectors.append(vector)
    top_topics = []
    for vector in vectors:
        top_topic = sorted(vector, key = lambda x: x[1], reverse=True)[0]
        top_topics.append(top_topic)
    topic_numbers = [topic[0] if (abs(topic[1] - (1/num_topics)) >= .01) else -1 for topic in top_topics]
    return topic_numbers

In [28]:
def add_jeremy_features(row):
    text = row.text
    doc = nlp(text)
    lemmas = list()
    entities = list()
    for token in doc:
        if token.text == ':':
            row['has_colon'] = 1
        if token.text == ';':
            row['has_semicolon'] = 1
        if token.text == '-':
            row['has_dash'] = 1
        pos = token.pos_
        row[pos] += 1
        if token.is_stop or not token.is_alpha:
            continue
        lemma = token.lemma_.strip().lower()
        if lemma:
            lemmas.append(lemma)
    for ent in doc.ents:
        entities.append(ent.text)
    lemmas = ' '.join(lemmas)
    blob = TextBlob(text)
    row['subjectivity'] = blob.sentiment.subjectivity
    row['polarity'] = blob.sentiment.polarity
    row['starts_conj'] = int(doc[0].pos_ == 'CONJ')
    row['ends_prep'] = int(doc[0].pos_ == 'PREP')
    row['entities'] = entities
    row['lemmas'] = lemmas
    row['raw_text_length'] = len(text)
    row['num_words'] = len(doc)
    row['avg_word_len'] = row.raw_text_length / row.num_words
    row['vector_avg'] = np.mean(nlp(lemmas).vector)
    return row


In [30]:
df = df.apply(lambda x: add_features(x), axis=1)

19578

In [43]:
def rhyme_frequency(text):
    words = list(set([i.lower() for i in text.split(' ')]))
    num_words = len(words)
    num_rhymes = 0
    for i in words:
        other = copy.copy(words)
        other.remove(i)
        rhymes = pronouncing.rhymes(i)
        matches = set(other).intersection(set(rhymes))
        num_rhymes += len(matches)/2
    return num_rhymes/num_words

def add_remy_features(df):
    text = df.text.to_list()
    nopunct_text = []
    for i in text:
        np = re.sub(r'[^\w\s]','',i)
        nopunct_text.append(np)
    df['NoPunct'] = nopunct_text
    df['whom']=df.NoPunct.apply(lambda x: 1 if ('whom ' in x.lower()) else 0)
    df['ing']=df.NoPunct.apply(lambda x: len([i for i in x.split(' ') if i[-3:]=='ing'])/len(x.split(' ')))
    df['pluperfect']= df.text.apply(lambda x: 1 if ' had ' in x else 0)
    df['rhyme_frequency'] = df.text.apply(lambda x: rhyme_frequency(x))
    df['dale_chall'] = df.text.apply(lambda x: textstat.dale_chall_readability_score(x))
    df['fleisch_reading_ease'] = df.text.apply(lambda x: textstat.flesch_reading_ease(x))
    df['lexicon'] = df.text.apply(lambda x: textstat.lexicon_count(x))
    df['word_diversity'] = df.lexicon/df.num_words
    df.FleischReadingEase = df.fleisch_reading_ease-df.fleisch_reading_ease.min()
    df.drop('NoPunct',axis=1,inplace=True)
    return df

In [45]:
let_our_powers_combine = add_remy_features(df)